In [8]:
import mimetypes
import pathlib
import shutil
import tempfile

from PIL import Image, ImageOps
from pillow_heif import register_heif_opener

register_heif_opener()


In [9]:
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / "data"
INPUTS_DIR = DATA_DIR / "inputs"
READY_DIR = DATA_DIR / "ready"
OUTPUTS_DIR = DATA_DIR / "outputs"

In [ ]:
def perform_clear_and_optimize_image(image_path, output_path, max_size=(1920, 1920)):
    """
    Removes all metadata from an image (e.g. EXIF data).
    Optimizes the image file size while preserving quality and transparency when needed.
    """
    # Convert to Path objects
    image_path = pathlib.Path(image_path)
    output_path = pathlib.Path(output_path)
    
    # Open and create clean copy
    original = Image.open(image_path)

    # Determine if image has transparency
    has_transparency = (
        original.mode in ('RGBA', 'LA') or 
        (original.mode == 'P' and 'transparency' in original.info)
    )
    
    # Auto-rotate based on EXIF
    original = ImageOps.exif_transpose(original)

    # Resize if larger than max_size while maintaining aspect ratio
    if original.size[0] > max_size[0] or original.size[1] > max_size[1]:
        original.thumbnail(max_size, Image.Resampling.LANCZOS)

    # Convert mode based on transparency
    if has_transparency:
        if original.mode != 'RGBA':
            original = original.convert('RGBA')
        best_format = 'PNG'
    else:
        if original.mode in ('RGBA', 'P', 'LA'):
            original = original.convert('RGB')
        best_format = 'JPEG'

    # Save with optimized settings
    save_kwargs = {}
    if best_format == 'JPEG':
        save_kwargs.update({
            'quality': 85,
            'optimize': True,
            'progressive': True
        })
        output_path = output_path.with_suffix('.jpg')
    elif best_format == 'PNG':
        save_kwargs.update({
            'optimize': True,
            'compress_level': 6
        })
        output_path = output_path.with_suffix('.png')
    print(f'Saving {output_path}')
    original.save(output_path, format=best_format, **save_kwargs)
    return output_path

In [ ]:
def perform_is_image(path, require_can_open=True):
    try:
        guessed_type, encoding = mimetypes.guess_type(path)
    except:
        guessed_type = ""
    guessed_image = "image" in guessed_type
    if not guessed_image:
        return False
    if guessed_image and require_can_open:
        try:
            img_ = Image.open(path)
        except:
            return False
    return True


In [14]:
image_file_paths = []

for file_path in INPUTS_DIR.glob("*"):
    is_image = perform_is_image(file_path)
    if not is_image:
        continue
    image_file_paths.append(file_path)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3088x2316 at 0x139F72710>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1582x2792 at 0x13A1805F0>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1546x2482 at 0x13A1805F0>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=2102x2648 at 0x13A1805F0>
<PIL.MpoImagePlugin.MpoImageFile image mode=RGB size=4032x3024 at 0x139F72710>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3088x2316 at 0x139F72710>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1990x2532 at 0x13A180830>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=2786x2028 at 0x13A180830>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=3284x1950 at 0x13A180830>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=2212x2566 at 0x13A180830>
<PIL.MpoImagePlugin.MpoImageFile image mode=RGB size=3088x2316 at 0x139F72710>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3088x2316 at 0x139F72710>
<PIL.MpoImagePlugin.MpoImageFile image 

In [15]:
image_file_paths

[PosixPath('/Users/cfe/Dev/superme-api/data/inputs/1.jpeg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/8.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/9.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/14.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/6.jpeg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/7.jpeg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/12.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/13.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/11.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/10.png'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/4.jpeg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/5.jpeg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/2.jpeg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/inputs/3.jpg')]

In [19]:
OUTPUTS_DIR.mkdir(exist_ok=True, parents=True)

In [20]:
zip_outpath = OUTPUTS_DIR / "images.zip"
zip_outpath.exists()

False

In [23]:
zip_outpath.with_suffix('')

PosixPath('/Users/cfe/Dev/superme-api/data/outputs/images')

In [26]:
with tempfile.TemporaryDirectory() as temp_dir:
    for path in image_file_paths:
        shutil.copy(path, temp_dir)
    shutil.make_archive(zip_outpath.with_suffix(''), 'zip', temp_dir)